In [1]:
!pip install census

In [2]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

# Census API Key
from config import api_key
#c = Census(api_key, year=2016)

In [65]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
def get_census(year):
    c = Census(api_key, year=year)
    census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E"), 
                         {'for': 'county:*'})
                        # {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
    census_pd = pd.DataFrame(census_data)

# Column Reordering
    census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "county": "County"
                                      })

# Add in Poverty Rate (Poverty Count / Population)
    census_pd["Poverty Rate"] = 100 * \
        census_pd["Poverty Count"].astype(
            int) / census_pd["Population"].astype(int)

# Final DataFrame
    census_pd = census_pd[["Name","County", "Population", "Median Age", "Household Income",
                           "Per Capita Income", "Poverty Count", "Poverty Rate"]]

    census_pd['county_name'] = census_pd['Name'].str.replace(r" County,(.*)",'').str.upper()
    census_pd['state'] = census_pd['Name'].str.replace(r"(.*) County, ",'').str.upper()
# Visualize
    
    return census_pd

census_2016 = get_census(2016)
census_2016.head()
census_2016['year'] = 2016

census_2020 = get_census(2020)
census_2020['year'] = 2020
census_2020.head()
census = pd.concat([census_2016,census_2020])

C:\Users\Sridevi\AppData\Local\Temp/ipykernel_15300/1925853534.py:33: FutureWarning: The default value of regex will change from True to False in a future version.
  census_pd['county_name'] = census_pd['Name'].str.replace(r" County,(.*)",'').str.upper()
C:\Users\Sridevi\AppData\Local\Temp/ipykernel_15300/1925853534.py:34: FutureWarning: The default value of regex will change from True to False in a future version.
  census_pd['state'] = census_pd['Name'].str.replace(r"(.*) County, ",'').str.upper()


In [66]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
winners_pd=pd.read_csv("countypres_2000-2020.csv")
winners_pd = winners_pd[((winners_pd['year'] == 2016) | (winners_pd['year'] == 2020)) & (winners_pd['mode'] == 'TOTAL')]
dem_pd = winners_pd[winners_pd['party'] == 'DEMOCRAT']
dem_pd
 #& ((winners_pd['party'] == 'DEMOCRAT') | (winners_pd['party'] == 'REPUBLICAN') )
rep_pd = winners_pd[winners_pd['party'] == 'REPUBLICAN']
rep_pd

merge_pd=dem_pd.merge(rep_pd, on = ['year','state', 'county_name'])

merge_pd[merge_pd['county_name']=='BALDWIN']
dem_pd
#rep_pd
merge_pd['winner'] = np.where(merge_pd['candidatevotes_x'] > merge_pd['candidatevotes_y'], merge_pd['party_x'], merge_pd['party_y'])
merge_pd = merge_pd[['year','state','county_name','winner']]
merge_pd

,year,state,county_name,winner
0,2016,ALABAMA,AUTAUGA,REPUBLICAN
1,2016,ALABAMA,BALDWIN,REPUBLICAN
2,2016,ALABAMA,BARBOUR,REPUBLICAN
3,2016,ALABAMA,BIBB,REPUBLICAN
4,2016,ALABAMA,BLOUNT,REPUBLICAN
...,...,...,...,...
5469,2020,WYOMING,SWEETWATER,REPUBLICAN
5470,2020,WYOMING,TETON,DEMOCRAT
5471,2020,WYOMING,UINTA,REPUBLICAN
5472,2020,WYOMING,WASHAKIE,REPUBLICAN


In [71]:
merge_pd2 = merge_pd.merge(census, on=['year','state', 'county_name'])

merge_pd2 = merge_pd2[['year','state', 'county_name','Population','Median Age', 'Household Income', 'Per Capita Income', 'Poverty Rate', 'winner']]
merge_pd2

,year,state,county_name,Population,Median Age,Household Income,Per Capita Income,Poverty Rate,winner
0,2016,ALABAMA,AUTAUGA,55049.0,37.8,53099.0,26168.0,12.165525,REPUBLICAN
1,2016,ALABAMA,BALDWIN,199510.0,42.3,51365.0,28069.0,12.806877,REPUBLICAN
2,2016,ALABAMA,BARBOUR,26614.0,38.7,33956.0,17249.0,23.427519,REPUBLICAN
3,2016,ALABAMA,BIBB,22572.0,40.2,39776.0,18988.0,15.018607,REPUBLICAN
4,2016,ALABAMA,BLOUNT,57704.0,40.8,46212.0,21033.0,16.361084,REPUBLICAN
...,...,...,...,...,...,...,...,...,...
5206,2020,WYOMING,SWEETWATER,43352.0,35.7,73384.0,34282.0,11.328197,REPUBLICAN
5207,2020,WYOMING,TETON,23356.0,40.3,87053.0,60331.0,6.499401,DEMOCRAT
5208,2020,WYOMING,UINTA,20374.0,36.0,72458.0,30442.0,9.472858,REPUBLICAN
5209,2020,WYOMING,WASHAKIE,7933.0,42.6,57306.0,30212.0,10.260935,REPUBLICAN
